# Evaluation Scenarios

In [28]:
using Revise
using Random
using Printf
using StaticArrays

# POMDP and learning
using POMDPs
using BeliefUpdaters
using POMDPPolicies
using POMDPSimulators
using POMDPModelTools
using MDPModelChecking
using LocalApproximationValueIteration
using DeepRL
using DeepQLearning
using Flux

# Driving related Packages
using AutomotiveDrivingModels
using AutomotiveSensors
using AutomotivePOMDPs
using PedCar

# Visualization
using AutoViz
using Reel
using ProgressMeter

In [29]:
include("../src/masking.jl")
include("../src/util.jl")
include("../src/masked_dqn.jl")
include("../src/qmdp_approximation.jl")
include("../src/render_helpers.jl")

animate_history (generic function with 1 method)

**Environment parameters**

In [30]:
rng = MersenneTwister(1);
cam =StaticCamera(VecE2(0., -8.), 14.0);

In [31]:
params = UrbanParams(nlanes_main=1,
                     crosswalk_pos =[VecSE2(6, 0., pi/2), VecSE2(-6, 0., pi/2), VecSE2(0., -5., 0.)],
                     crosswalk_length =  [14.0, 14., 14.0],
                     crosswalk_width = [4.0, 4.0, 3.1],
                     stop_line = 22.0)
env = UrbanEnv(params=params);

In [107]:
pomdp = UrbanPOMDP(env=env,
                   sensor = PerfectSensor(),
                   ego_goal = LaneTag(2, 1),
                   max_cars=0, 
                   max_peds=0, 
                   car_birth=0., 
                   ped_birth=0., 
                   max_obstacles=0, # no fixed obstacles
                   lidar=false,
                   ego_start=20,
                   ΔT=0.1);

In [33]:
const TURN_RIGHT = SVector(LaneTag(3,1), LaneTag(5,1))
const STRAIGHT_FROM_RIGHT = SVector(LaneTag(3,1), LaneTag(4,1));

In [79]:
const RIGHT_OBSTACLE = ConvexPolygon([VecE2(8.125, -7.500), VecE2(26.875, -7.500), VecE2(26.875, -3.000), VecE2(8.125, -3.000)], 4)
const LEFT_OBSTACLE = ConvexPolygon([VecE2(-26.875, -7.500),VecE2(-8.125, -7.500),VecE2(-8.125, -3.000),VecE2(-26.875, -3.000)], 4);

## Evaluation Policy

In [ ]:

# Set belief updater
updater = NothingUpdater()

# Set policy
policy = FunctionPolicy(s -> UrbanAction(0.)) # constant policy


## Case study 1: Perception Error

It is difficult to handle perception error in rule-based policies. The POMDP framework, through the use of belief state, is well suited to address perception errors like sensor noise, false positive, false negative or sensor occlusion. 

**Scenario 1: Make some noise!**
Regular intersection, with cars and pedestrian, no obstacles. We are only looking at how sensor noise is taken care of.

In [84]:
# Set car parameter
car_on = true
car_s0 = 0.
car_v0 = 0.
car_posF = Frenet(env.roadway[LaneTag(3, 1)], car_s0)
car = Vehicle(VehicleState(car_posF, env.roadway, car_v0), pomdp.car_type, 2)

# Set pedestrian parameter
ped_on = true
ped_s0 = 0.
ped_v0 = 0.
ped_posF = Frenet(env.ped_roadway[LaneTag(18,1)], ped_s0) # choose between 17, 18, 19
ped = Vehicle(VehicleState(ped_posF, env.roadway, ped_v0), pomdp.ped_type, 101)

# Set sensor characteristics
pomdp.sensor = GaussianSensor(pos_noise = LinearNoise(min_noise=0.5, increase_rate=0.1),
                              vel_noise = LinearNoise(min_noise=0.5, increase_rate=0.1),
                              false_positive_rate = 0.1,
                              false_negative_rate = 0.1)

# Initial state
function initialize_scenario(pomdp::UrbanPOMDP, car::Vehicle, ped::Vehicle)
    s0 = Scene()

    if car_on
        push!(s0, car)
        pomdp.max_cars = 1
        pomdp.models[2] = pomdp.car_models[TURN_RIGHT]
    end
    if ped_on
        push!(s0, ped)
        pomdp.max_peds = 1
        pomdp.models[101] = IntelligentPedestrian(dt = pomdp.ΔT, crosswalk=get_lane(env.roadway, ped), conflict_lanes=get_conflict_lanes(get_lane(env.roadway, ped), env.roadway))
    end

    push!(s0, initial_ego(pomdp, rng))
    return s0
end
s0 = initialize_scenario(pomdp, car, ped)

# Initial belief 
b0 = nothing

hr = HistoryRecorder(max_steps=200, rng=rng)
@time hist = POMDPSimulators.simulate(hr, pomdp, policy, updater, b0, s0);

  0.189078 seconds (203.00 k allocations: 14.045 MiB)


In [39]:
animate_history(hist, pomdp,
                obs_overlays = o -> [GaussianSensorOverlay(sensor=pomdp.sensor, o=[veh for veh in obs_to_scene(pomdp, o) if veh.id != EGO_ID], color=MONOKAI["color2"])],
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                speed_factor = 2,
                cam =  cam)

Frames{MIME{Symbol("image/png")}}("/tmp/tmpPrBHJq", 0x00000000000000c8, 20.0, nothing)

┌ Warning: /mnt/c/Users/Maxime/wsl/.julia/packages/DeepQLearning/EeLUa/src is not an existing directory, Revise is not watching
└ @ Revise /mnt/c/Users/Maxime/wsl/.julia/packages/Revise/TmjcT/src/Revise.jl:361
┌ Warning: /mnt/c/Users/Maxime/wsl/.julia/packages/DeepRL/ynDbA/src is not an existing directory, Revise is not watching
└ @ Revise /mnt/c/Users/Maxime/wsl/.julia/packages/Revise/TmjcT/src/Revise.jl:361


**Scenario 2: Blind Corners**
A similar scenario than above, with sensor occlusions. 

In [87]:
pomdp.max_obstacles = 1
env.obstacles = [LEFT_OBSTACLE]

s0 = initialize_scenario(pomdp, car, ped)
# Initial belief 
b0 = nothing

hr = HistoryRecorder(max_steps=200, rng=rng)
@time hist = POMDPSimulators.simulate(hr, pomdp, policy, updater, b0, s0);

  0.087423 seconds (125.13 k allocations: 10.076 MiB)


In [88]:
animate_history(hist, pomdp,
                obs_overlays = o -> [GaussianSensorOverlay(sensor=pomdp.sensor, o=[veh for veh in obs_to_scene(pomdp, o) if veh.id != EGO_ID], color=MONOKAI["color2"])],
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                speed_factor = 2,
                cam =  cam)

Frames{MIME{Symbol("image/png")}}("/tmp/tmpHafC9Q", 0x00000000000000c8, 20.0, nothing)

## Case Study 2: Interactions

The POMDP approach can capture interaction between other traffic participants such as a vehicle yielding for a pedestrian or letting the right of way to our vehicle

**Scenario 1: Vehicle - Pedestrian interaction**

In [108]:
# Set car parameter
car_on = true
car_s0 = 0.
car_v0 = 0.
car_posF = Frenet(env.roadway[LaneTag(1, 1)], car_s0)
car = Vehicle(VehicleState(car_posF, env.roadway, car_v0), pomdp.car_type, 2)

# Set pedestrian parameter
ped_on = true
ped_s0 = 2.
ped_v0 = 0.
ped_posF = Frenet(env.ped_roadway[LaneTag(17,1)], ped_s0) # choose between 17, 18, 19
ped = Vehicle(VehicleState(ped_posF, env.roadway, ped_v0), pomdp.ped_type, 101)

# Sensor 
pomdp.sensor = PerfectSensor()

s0 = initialize_scenario(pomdp, car, ped)

# Initial belief 
b0 = nothing

hr = HistoryRecorder(max_steps=200, rng=rng)
@time hist = POMDPSimulators.simulate(hr, pomdp, policy, updater, b0, s0);

  0.091084 seconds (59.66 k allocations: 6.383 MiB)


In [109]:
animate_history(hist, pomdp,
                step_overlays = s -> [TextOverlay(text = ["step: $s"], font_size=20, pos=VecE2(pomdp.env.params.x_min + 3.,4.), incameraframe=true)],
                extra_overlays = [IDOverlay()],
                speed_factor = 2,
                cam =  cam)

Frames{MIME{Symbol("image/png")}}("/tmp/tmpOfQwmR", 0x00000000000000c8, 20.0, nothing)

**Scenario 2: Vehicle - Vehicle interactions**

In [ ]:
# Set car parameter
car_on = true
car_s0 = 0.
car_v0 = 0.
car_posF = Frenet(env.roadway[LaneTag(1, 1)], car_s0)
car = Vehicle(VehicleState(car_posF, env.roadway, car_v0), pomdp.car_type, 2)

## Case Study 3: Scalability

Our approach scales to multiple cars, pedestrians and obstacles. Conventional rule-based approach are very difficult to design in such scenarios since there are a lot of cases to take into account.